In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
class CorralationAnalysis:
    def __init__(self, dataframe):
        if not isinstance(dataframe, pd.DataFrame):
            raise ValueError("Input must be a Pandas DataFrame.")
        self.dataframe = dataframe

    def run_regression(self, target, predictors):
        X = self.dataframe[predictors]
        y = self.dataframe[target]
        X_const = sm.add_constant(X)
        model = sm.OLS(y, X_const).fit()
        results = {
            "Coefficients": model.params.to_dict(),
            "P_values": model.pvalues.to_dict(),
            "R_squared": model.rsquared,
            "Adjusted_R_squared": model.rsquared_adj,
            "Summary": model.summary()
        } 
        return results

    def calculate_vif(self, predictors):  
        X = self.dataframe[predictors] 
        X_const = sm.add_constant(X)  
        vif_data = pd.DataFrame()  
        vif_data["Variable"] = predictors 
        vif_data["VIF"] = [
            variance_inflation_factor(X_const.values, i + 1)
            for i in range(len(predictors))
        ] 
        return vif_data

    def rotational_analysis(self, group_columns):
        results = []
        for target in group_columns:
            predictors = [col for col in group_columns if col != target]
            regression_result = self.run_regression(target, predictors)
            vif_result = self.calculate_vif(predictors)
            print("\r")
            results.append({
                "Target": target,
                "Predictors": predictors,
                "Regression": regression_result,
                "VIF": vif_result
            })
        return results
    
    def generate_summary_table(self, group_columns):
      summary_data = []  
      analysis_results = self.rotational_analysis(group_columns) 

      for result in analysis_results:
        target = result["Target"] 
        predictors = result["Predictors"]
        p_values = result["Regression"]["P_values"]

        target_vif_values = [] 
        target_p_values = [] 

        for predictor in predictors:
            vif_result = self.calculate_vif(predictors)
            vif_value = vif_result.loc[vif_result["Variable"] == predictor, "VIF"].iloc[0]
            p_value = p_values[predictor]

            target_vif_values.append(vif_value)
            target_p_values.append(p_value)
            
            summary_data.append({
                "Target": target,
                "Predictor": predictor,
                "P_value": p_value,
                "VIF": vif_value
            }) 
        
        mean_vif = np.mean(target_vif_values) 
        mean_p_value = np.mean(target_p_values) 

        summary_data.append({
            "Target": target,
            "Predictor": "Mean (All Predictors)",
            "P_value": mean_p_value,
            "VIF": mean_vif
        }) 
    
      return pd.DataFrame(summary_data)
    